# 파인튜닝을 위한 환경 및 라이브러리 셋업

## 필요 라이브러리
- datasets : 오디오나 컴퓨터비전, 자연어 처리 작업을 위한 데이터셋을 쉽게 엑세스하고 공유할 수 있는 라이브러리
- bitsandbytes : cuda에서 제공하는 함수. 8비트, 4비트 양자화를 위한 라이브러리.
    - 양자화(quantization)
        - 모델의 성능과 효율성 향상을 위해 Deep Learning의 weight, activation function 출력을 더 작은 비트로 표현하도록 변환하는 기술
        - 모델 크기 감소, 계산 속도 향상, 메모리 사용량을 감소하여 효율적인 모델 배포와 실행을 가능하게 해주는 중요한 방법
        - 채널 수가 줄어드는 만큼 손실 발생
- peft : Parameter-Efficient Fine-Tuning. 적은수의 파라미터만 파인튜닝.
- accelerate : GPU 가속기 
- Wandb : 훈련을 하면서 발생할 수 있는 로그들 볼 수 있는 툴.
- sentense trainsformers : text, embeddings 라이브러리

## Trainer, Config, Argument
- Trainer : PyTorch에서 제공하는 라이브러리. 모델이 훈련되는 방식을 사용자가 정의해서 사용. Seq2SeqTrainer, Seq2SeqTrainingArguments를 많이 사용함
    - TRL : tranformer 언어 모델 관련 풀스택 라이브러리
        - Model Classes
        - SFTTrainer
        - RewardTrainer
        - PPOTrainer
        - Best-of_N sampling
        - DPOTrainer
        - TextEnvironment
    - STRTrainer : Supervised Fine-tuning Trainer
    - DPOTrainer : Data Colloection, Optimization
    - ORPO Trainer


In [ ]:
!pip install transformers accelerate huggingface_hub

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import transformers
import torch

model_id="meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages=[
    {"role": "system", "content" : "You are a helpful assistant."},
    {"role": "user", "content": "Who are you?"},
]

"""
terminators = [
  pipeline.tokenzier.eos_token,
  pipeline.tokenizer.convert_tokens_to_string(["<|eot_id|>])
]
"""

eos_token_id = pipeline.tokenizer.eos_token_id

outputs=pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(outputs[0]["generated_text"])

In [ ]:
# T4로도 성능이 안나와서 다른 유형 선택해야함
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id="meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who are you?"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

terminators = [
    tokenizer.eos_token_id,

]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

# Fine Tuning PEFT LoRA 이용

In [ ]:
!pip install llama-recipes ipywidgets # ipywidgets : 노트북과 ipython 커널을 위한 대화형 html 위젯

!pip install -q llama-recipies
!pip install -q huggingface_hub

## Step 1: 모델 로드

In [ ]:
import torch
from transformers import LlamaForCausalLM, AutoTokenizer
from llama_recipes.configs import train_config as TRAIN_CONFIG

train_config = TRAIN_CONFIG()
train_config.model_name = "meta-llama/Meta-Llama-3-8B"
train_config.num_epochs = 1                             # epoch 수
train_config.run_validation = False                     # 학습 과정에서 validation 수행 여부
train_config.gradirent_accumulation_steps=4             # 그라이언트 누적 스텝 수
train_config.batch_size_training=1                      # 학습 시 사용하는 배치 크기
train_config.l4=3e-4                                    # 학습률
train_config.use_fast_kernels=True                      # Fast Kenrnel 사용 여부
train_config.use_fp16=True                              # FP16 연산 사용 여부
train_config.context_length=1024 if torch.cuda.get_device_properties(0).total_memory<16e9 else 2024 # T4 16GB or A10 24GB
train_config.batching_strategy = "packing"              # 배치 전략
train_config.output_dir = "/content/drive/MyDrive/gdrive/llama_result"

from transformers import BitsAndBytesConfig             # 8비트 양자화
config = BitsAndBytesConfig(
    load_in_8biyt=True
)

model = LlamaForCausalLM.from_pretrained(
    train_config.model_name,
    quantization_config=config,
    device_map="auto",
    use_cache=False,
    attn_implementation="sdpa" if train_config.use_fast_kernels else None,
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
tokenizer.pad_token = tokenizer.eos_token



## Step 2: 모델 체크


In [ ]:
# 요약 요청 프롬프트 
# 단점 : 아직 영어만 가능함
eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-)
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")    # 토크나이저를 사용하여 텍스트 데이터를 토크나이징, pytorch텐서로 변환해서 gpu로 이동

model.eval()                                                              # 모델을 평가 모드로 설정
with torch.no_grad():                                                     # 그라디언트 계산을 비활성화. 메모리 사용량을 줄이고 연산속도 향상. 평가 및 추론 시 사용
  print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

## Step 3: 데이터셋 로드

In [ ]:
from llama_recipes.configs.datasets import samsum_dataset
from llama_recipes.data.concatenator import ConcatDataset
from llama_recipes.utils.config_utils import get_dataloader_kwargs
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset

samsum_dataset.trust_remote_code=True
train_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')                  # 전처리된 데이터셋 생성

train_dl_kwargs = get_dataloader_kwargs(train_config, train_dataset, tokenizer, "train")      # 데이터로더 설정

if train_config.batching_strategy == "packing":                                               # 패킹 전략 사용
        train_dataset = ConcatDataset(train_dataset, chunk_size=train_config.context_length)

# Create DataLoaders for the training and validation dataset
train_dataloader = torch.utils.data.DataLoader(                                               # 데이터 로더 생성
    train_dataset,
    num_workers=train_config.num_workers_dataloader,
    pin_memory=True,
    **train_dl_kwargs,
)

## Step 4: PEFT를 위한 모델 준비

In [ ]:
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig
from dataclasses import asdict
from llama_recipes.configs import lora_config as LORA_CONFIG

lora_config = LORA_CONFIG()
lora_config.r = 8 # low rank size
lora_config.lora_alpha = 31 # LoRA의 스케일링 벡터
lora_dropout: float=0.01    # 드롭아웃 비율 0.01

peft_config = LoraConfig(**asdict(lora_config)) # PEFT 설정, asdic: lora_config 객체를 딕셔너리 형태로 변환

model = prepare_model_for_kbit_training(model) # model을 k-bit 훈련으로 준비, 모델의 특정 부분을 조정하여 k-bit 정밀도를 사용하도록 설정
model = get_peft_model(model, peft_config) # PEFT 모델 생성

## Step 5: 모델 파인 튜닝

In [ ]:
import torch.optim as optim
from llama_recipes.utils.train_utils import train
from torch.optim.lr_scheduler import StepLR

model.train()

optimizer = optim.AdamW(                                    # 옵티마이저 설정
            model.parameters(),
            lr=train_config.lr,
            weight_decay=train_config.weight_decay,
        )
scheduler = StepLR(optimizer, step_size=-1, gamma=train_config.gamma)


results = train(
    model,
    train_dataloader,
    None,
    tokenizer,
    optimizer,
    scheduler,
    train_config.gradient_accumulation_steps,
    train_config,
    None,
    None,
    None,
    wandb_run=None
)

## Step 6 CheckPoint 생성
- 트레이닝 중간 체크 포인트 파일 생성

In [ ]:
model.save_pretrained(train_config.output_dir+"meta-llama-wonik")

## Step 7 모델 학습 확인

In [ ]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

In [ ]:
model.push_to_hub("wonik-hi/meta_llam_peft", use_auth_token=True)